In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed, GPT2Model, GPT2LMHeadModel, GPT2Tokenizer

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Define the model name
model_name = 'gpt2'  # 137M parameters

# Redownload the model and tokenizer
model1 = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# First class of prompt: city-country and person correspondence
# prompt = "alice - france,\nalice ~ paris,\nbob - germany,\nbob ~ berlin,\njohn - usa,\njohn ~"
# prompt = "Alice lives in France, Paris - Alice, John lives in Germany, Berlin - John, Peter lives in USA, Washington -"

# works even with some counterfactual prompts 
# prompt = "alice lives in france,\nparis - alice,\nbob lives in germany,\nberlin - bob,\njohn lives in usa,\nwashington -"
# prompt = "alice lives france,\nparis - alice,\nbob lives germany,\nberlin - bob,\njohn lives USA,\nwashington -"
# prompt = "alice resides france,\nparis - alice,\nbob resides germany,\nberlin - bob,\njohn resides usa,\nwashington -"
# prompt = "alice resides in france,\nparis - alice,\nbob resides in germany,\nberlin - bob,\njohn resides in usa,\nwashington -"
# prompt = "alice works france,\nparis - alice,\nbob works germany,\nberlin - bob,\njohn works usa,\nwashington -"
# prompt = "alice loves france,\nparis - alice,\nbob loves germany,\nberlin - bob,\njohn loves usa,\nwashington -"
# prompt = "alice drinks france,\nparis - alice,\nbob drinks germany,\nberlin - bob,\njohn drinks usa,\nwashington -"
# prompt = "alice skyscraper france,\nparis - alice,\nbob skyscraper germany,\nberlin - bob,\njohn skyscraper usa,\nwashington -"

# testing the working prompt - not working
# prompt = "alice is french,\nparis - alice,\nbob is german,\nberlin - bob,\njohn is american,\nwashington -"
# prompt = "alice ~ france,\nparis - alice,\nbob ~ germany,\nberlin - bob,\njohn ~ usa,\nwashington -"
# prompt = "alice is from france,\nparis - alice,\nbob is from germany,\nberlin - bob,\njohn is from usa,\nwashington -"
# prompt = "alice travels france,\nparis - alice,\nbob travels germany,\nberlin - bob,\njohn travels usa,\nwashington -"
# prompt = "alice cuts france,\nparis - alice,\nbob cuts germany,\nberlin - bob,\njohn travels cuts usa,\nwashington -"
# prompt = "alice drills france,\nparis - alice,\nbob drills germany,\nberlin - bob,\njohn drills cuts usa,\nwashington -"

# corrupted prompt 

prompt = "Alice lives in France, Paris - Alice, John lives in Germany, Berlin - John, Peter lives in Italy, Paris -"

# prompt = "alice lives in france,\nparis - alice,\nbob lives in germany,\nberlin - bob,\njohn lives in usa,\nbucharest -"
# prompt = "alice france, paris alice, bob germany, berlin bob, john usa, washington"
# prompt = "Alice is from France. The capital of France is Paris.\nBob is from Germany. The capital of Germany is Berlin.\nJohn is from the USA. The capital of the USA is"
# prompt = "Alice is from France, and she lives in Paris.\nBob is from Germany, and he lives in Berlin.\nJohn is from USA"


# Tokenize the input prompt
encoded_input = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = encoded_input.input_ids
print(f"Length of tokens: {len(input_ids[0])}")
attention_mask = encoded_input.attention_mask

# Generate text
gen_tokens = model1.generate(
    input_ids,
    attention_mask=attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.01,
    max_length=input_ids.size(1) + 2,
)

# Decode the generated tokens to text
gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print("Generated Text: ", gen_text)

Length of tokens: 25
Generated Text:  Alice lives in France, Paris - Alice, John lives in Germany, Berlin - John, Peter lives in Italy, Paris - Peter,


In [27]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model2 = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")

text = "alice france, alice paris, bob germany, bob berlin, john usa, john "
encoded_input = tokenizer(text, return_tensors='pt')
output = model2(**encoded_input)

In [28]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Jupiter is the 5th planet from the Sun.\nSaturn is the 6th planet from the Sun.\nUranus is the 7th planet from the Sun.\nNeptune is the 8th planet from the Sun.\nMercury is the 1st planet from the Sun.\nVenus is", max_length=70, num_return_sequences=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Jupiter is the 5th planet from the Sun.\nSaturn is the 6th planet from the Sun.\nUranus is the 7th planet from the Sun.\nNeptune is the 8th planet from the Sun.\nMercury is the 1st planet from the Sun.\nVenus is the 2nd planet from'}]